In [ ]:
import torch
import xarray as xr
import matplotlib.pyplot as plt


# Load EddyNet with pre-trained weights and send to GPU if available
model = torch.hub.load("edwinytgoh/eddynet", "eddynet", pretrained=True, num_classes=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Obtain filtered ADT data as a numpy array
!wget https://raw.githubusercontent.com/edwinytgoh/eddynet/master/data/dt_global_twosat_phy_l4_20220428_vDT2021.nc
ds = xr.open_dataset("dt_global_twosat_phy_l4_20220428_vDT2021.nc")
adt = ds["adt"].values.squeeze()

# Convert ADT array to PyTorch tensor with batch dimension and channel dimension, each of size 1
adt_tensor = torch.from_numpy(adt).reshape(1, 1, *adt.shape).float().to(device)
adt_tensor[torch.isnan(adt_tensor)] = 0  # Set nan land masks to 0

# Feed ADT tensor into EddyNet and obtain eddy mask
logits = model(adt_tensor)
eddy_mask = torch.argmax(logits, dim=1).squeeze().cpu().numpy()

# Plot ADT and eddy masks
fig, ax = plt.subplots(2, 1, figsize=(10, 10))
im = ax[0].imshow(adt[::-1, ...])
ax[0].set_title("Absolute Dynamic Topography (ADT) from 04-28-2022")
cbar = plt.colorbar(im, ax=ax[0])

im = ax[1].imshow(eddy_mask[::-1, ...])
ax[1].set_title("EddyNet Detected Eddies on 04-28-2022")
cbar = plt.colorbar(im, ax=ax[1], ticks=[0, 1, 2])
cbar.ax.set_yticklabels(["Negative", "Anticyclonic", "Cyclonic"])
plt.tight_layout()